In [2]:
#BPR
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial
from tqdm import tqdm
%matplotlib inline

In [3]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
DATA_FOR_BPR = "data_for_BPR.csv"
SAVE_DIR = "../gamedata/train_result/"

In [ ]:
# #data preprocessing
# data = pd.read_csv(DATA_DIR+DATA_FILE)
# gameid = data['gameid'].drop_duplicates()
# gameid_list = []
# gameid_list = list(gameid)
# gameid2newid = {old:new+1 for new, old in enumerate(gameid_list)}
# data['new_game_id'] = data['gameid'].map(gameid2newid)

# events = data['eventname'].drop_duplicates()
# events_list = list(events)
# events2id = {old:new+1 for new,old in enumerate(events_list)}
# data['new_event_id'] = data['eventname'].map(events2id)

In [ ]:
#data.to_csv(DATA_DIR+'data_for_BPR.csv',index=False)

In [15]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
#data[data['new_game_id']==63]

In [4]:
def load_data(data_path):
    game_events = defaultdict(set)
    game_count = -1
    event_count = -1
    with open(data_path,'r') as f:
        header_line = next(f)
        for line in f.readlines():
            _, _, _, _, _, _, gameid, eventid = line.split(",")
            gameid = int(gameid)
            eventid = int(eventid)
            #print(u,i)
            game_events[gameid].add(eventid)
            game_count = max(gameid, game_count)
            event_count = max(eventid, event_count)
    print ("game_count:", game_count)
    print ("event_count:", event_count)
    #return max_u_id, max_i_id, user_ratings
    return game_count,event_count,game_events
    

#data_path = os.path.join('D:\\tmp\\ml-100k', 'u.data')
#user_count, item_count, user_ratings = load_data(data_path)
game_count,event_count,game_events_dict = load_data(DATA_DIR+DATA_FOR_BPR)

game_count: 1108
event_count: 11189


In [5]:
#
def generate_test(game_events_dict):
    user_test = dict()
    for u, i_list in game_events_dict.items():
        user_test[u] = random.sample(game_events_dict[u], 1)[0]
    return user_test

game_test_set = generate_test(game_events_dict)

In [6]:
#generate dataset for training
def generate_train_batch(game_events_dict, game_test_set, event_count, batch_size):
    t = []
    
    #print('generate batch..')
    for b in range(batch_size):
        break_flag1 = 0
        break_flag2 = 0
        u = random.sample(game_events_dict.keys(), 1)[0]
        #print('u:',u)
        i = random.sample(game_events_dict[u], 1)[0]
        while i == game_test_set[u]:
            break_flag1 += 1
            i = random.sample(game_events_dict[u], 1)[0]
            if break_flag1 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('i:',i)
        j = random.randint(1, event_count)
        while j in game_events_dict[u]:
            break_flag2 += 1
            j = random.randint(1, event_count)
            if break_flag2 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('j:',j)
        t.append([u, i, j])
        #print(u,i,j)
    #print('generate batch done!')
    return numpy.asarray(t)

#train_set = generate_train_batch(game_events_dict,game_test_set,event_count)

In [7]:
def generate_test_batch(game_events_dict, game_test_set, event_count):
    for u in game_events_dict.keys():
        t = []
        i = game_test_set[u]
        #for j in range(1, event_count+1):
        for j in range(1, 501):
            if not (j in game_events_dict[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

In [8]:
def bpr_mf(game_count, event_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [game_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [event_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

In [9]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(game_count, event_count, 20)
#     print(u, i, j, mf_auc, bprloss, train_op)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('start epoch')
    for epoch in range(1, 2):
        print ("epoch: ", epoch)
        _batch_bprloss = 0
        for k in tqdm(range(1, 5000)): # uniform samples from training set
            uij = generate_train_batch(game_events_dict, game_test_set,event_count,batch_size=128)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        games_count = 0
        _auc_sum = 0.0
        test_bprloss = 0.0
        # each batch will return only one user's auc
        for t_uij in generate_test_batch(game_events_dict, game_test_set, event_count):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            games_count += 1
            _auc_sum += _auc
            test_bprloss += _test_bprloss
#            print ("test_loss: ", test_bprloss/games_count)
        print ("test_loss: ", test_bprloss/games_count, "test_auc: ", _auc_sum/games_count)
        print ("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    saver.save(session, SAVE_DIR)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)
print('all done')

  0%|          | 3/4999 [00:00<02:52, 29.04it/s]

start epoch
epoch:  1


100%|██████████| 4999/4999 [00:22<00:00, 218.54it/s]


bpr_loss:  0.701012848198092
_train_op
test_loss:  0.7217065100205073 test_auc:  0.5117384662930702

Variable:  user_emb_w:0
Shape:  (1109, 20)
[[-0.16680019  0.06671733  0.07342078 ...  0.04041797 -0.0712643
  -0.09902354]
 [-0.06992461 -0.01237482 -0.02944826 ... -0.11615699 -0.00713816
   0.07623405]
 [-0.08501951  0.12530662 -0.10585739 ...  0.15839177  0.06824631
  -0.013723  ]
 ...
 [-0.1241233   0.1248052   0.07273874 ... -0.02480509 -0.08656926
  -0.01713629]
 [ 0.0432905  -0.01054744 -0.05510223 ... -0.20080622 -0.12606047
   0.26672494]
 [-0.02322073  0.03519825  0.1418342  ... -0.04313798  0.01445427
  -0.06659217]]
Variable:  item_emb_w:0
Shape:  (11190, 20)
[[-0.02066316 -0.013602    0.12544155 ... -0.00365485 -0.03126848
  -0.01378646]
 [ 0.04673948  0.08873983 -0.11176042 ... -0.06801496 -0.01756891
   0.03108914]
 [-0.04493034 -0.04996803  0.01145047 ...  0.0940714   0.18290472
   0.01742011]
 ...
 [ 0.07527296 -0.05969375 -0.15844385 ...  0.03059163  0.14614572
  -0.02

In [10]:
print(len(t_uij))
print(len(uij))

495
128


In [11]:
_test_bprloss

0.72082055

In [ ]:
saver.save(session, SAVE_DIR)

In [19]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][0], 0)
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)
result_1 = session1.run(u1_all)
print (result_1)

[[ 0.00287201 -0.0050352  -0.02212358 ... -0.07779539  0.03506517
  -0.06604174]]


In [ ]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [20]:
print("new recommedation ：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-10]] = 0
for index in range(len(p)):
    if p[index] != 0:
        select = data[data['new_event_id']==index].drop_duplicates('eventname')
  
        print (index, p[index],select['eventname'].values)

new recommedation ：
783 0.15230231 ['powerUsed']
1070 0.1581303 ['pushAcceptancePopupFail']
1247 0.1340827 ['campaignNodeCompleted']
1440 0.15474206 ['video_watched']
1889 0.14873128 ['arenaEnded']
6122 0.13872878 ['roomCreated']
6800 0.137376 ['comicCreated']
10366 0.14300333 ['trophyChallenge']
10577 0.15158731 ['ftueCallSignPromptEnteredName']
10984 0.15697959 ['uiFocusIn']


In [16]:
data[data['new_event_id']==11118]
data[data['new_game_id']==101].drop_duplicates('eventname')

,gameid,genre,platform,eventname,eventparameter,eventtype,new_game_id,new_event_id
82191,2976,No genre,pc,increaseXP,playerCoins,custom,101,1112
82192,2976,No genre,pc,clientDevice,deviceType,standard,101,9
82196,2976,No genre,pc,gameStarted,userLocale,standard,101,12
82202,2976,No genre,pc,chatReceiveMessage,userScore,custom,101,1113
82203,2976,No genre,pc,stageUnlocked,userScore,custom,101,1114
82204,2976,No genre,pc,testAborted,userScore,custom,101,1115
82205,2976,No genre,pc,block100Build,userScore,custom,101,1116
82206,2976,No genre,pc,challengeTimeout,userScore,custom,101,1117
82207,2976,No genre,pc,toolDashUse,userScore,custom,101,1118
82208,2976,No genre,pc,timeTrialNewRecord,userScore,custom,101,1119
